# Import libs

In [9]:

import matplotlib.pyplot as plt
import os
import numpy as np
# from models import *
import torch
import torch.optim
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
# from utils.denoising_utils import *
import utils
print(utils.__file__)
from utils import get_noisy_image
from utils import get_image, get_noise, np_to_torch
from skip import skip

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark =True
dtype = torch.cuda.FloatTensor

imsize =-1
PLOT = True
sigma = 25
sigma_ = sigma/255.

/home/yongqian/CV/CV-24Fall-Course-Project/my_dip/utils.py


ImportError: cannot import name 'get_noisy_image' from 'utils' (/home/yongqian/CV/CV-24Fall-Course-Project/my_dip/utils.py)

In [8]:
# deJPEG 
# fname = 'data/denoising/snail.jpg'

## denoising
fname = 'data/denoising/F16_GT.png'

# Load image

In [9]:
img_pil, img_np = get_image(fname, imsize)

if fname == 'data/denoising/snail.jpg':
    img_noisy_pil, img_noisy_np  = img_pil, img_np 
        
elif fname == 'data/denoising/F16_GT.png':
    # Add synthetic noise
    img_noisy_pil, img_noisy_np = get_noisy_image(img_np, sigma_)
    
else:
    assert False

# Setup

In [1]:

OPT_OVER = 'net'

reg_noise_std = 1./30. # set to 1./20. for sigma=50
LR = 0.01

OPTIMIZER='adam' # 'LBFGS'
show_every = 100
exp_weight=0.99

if fname == 'data/denoising/snail.jpg':
    num_iter = 2400
    input_depth = 3
    figsize = 5 
    
    net = skip(
                input_depth, 3, 
                num_channels_down = [8, 16, 32, 64, 128], 
                num_channels_up   = [8, 16, 32, 64, 128],
                num_channels_skip = [0, 0, 0, 4, 4], 
                upsample_mode='bilinear', downsample_mode='stride',
                need_sigmoid=True, need_bias=True, pad='reflection', act_fun='LeakyReLU')

    net = net.type(dtype)

elif fname == 'data/denoising/F16_GT.png':
    num_iter = 3000
    input_depth = 32 
    figsize = 4 

    net = skip(
                input_depth, n_channels = 3, 
                num_channels_down = [128]*5,
                num_channels_up =   [128]*5,
                num_channels_skip = [128]*5, 
                upsample_mode='bilinear', downsample_mode='stride',
                need_sigmoid=True, need_bias=True, pad='reflection', act_fun='LeakyReLU')
else:
    assert False
    
net_input = get_noise(input_depth, 'noise', (img_pil.size[1], img_pil.size[0])).type(dtype).detach()

# Compute number of parameters
s  = sum([np.prod(list(p.size())) for p in net.parameters()]); 
print ('Number of params: %d' % s)

# Loss
mse = torch.nn.MSELoss().type(dtype)

img_noisy_torch = np_to_torch(img_noisy_np).type(dtype)

NameError: name 'fname' is not defined

# Optimize

In [11]:
net_input_saved = net_input.detach().clone()
noise = net_input.detach().clone()
out_avg = None
last_net = None
psrn_noisy_last = 0

i = 0


log_dir = 'logs'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# 打开日志文件
log_file_path = os.path.join(log_dir, 'denoise_training_log.txt')
log_file = open(log_file_path, 'a')

def closure():
    
    global i, out_avg, psrn_noisy_last, last_net, net_input
    
    if reg_noise_std > 0:
        net_input = net_input_saved + (noise.normal_() * reg_noise_std)
    
    out = net(net_input)
    
    # Smoothing
    if out_avg is None:
        out_avg = out.detach()
    else:
        out_avg = out_avg * exp_weight + out.detach() * (1 - exp_weight)
            
    total_loss = mse(out, img_noisy_torch)
    total_loss.backward()
        
    
    psrn_noisy = compare_psnr(img_noisy_np, out.detach().cpu().numpy()[0]) 
    psrn_gt    = compare_psnr(img_np, out.detach().cpu().numpy()[0]) 
    psrn_gt_sm = compare_psnr(img_np, out_avg.detach().cpu().numpy()[0]) 
    
    # Note that we do not have GT for the "snail" example
    # So 'PSRN_gt', 'PSNR_gt_sm' make no sense
    # print ('Iteration %05d    Loss %f   PSNR_noisy: %f   PSRN_gt: %f PSNR_gt_sm: %f' % (i, total_loss.item(), psrn_noisy, psrn_gt, psrn_gt_sm), '\r', end='')
    log_message = ('Iteration %05d    Loss %f   PSNR_noisy: %f   PSRN_gt: %f PSNR_gt_sm: %f\n' % 
               (i, total_loss.item(), psrn_noisy, psrn_gt, psrn_gt_sm))
    # print(log_message, '\r', end='')
    log_file.write(log_message)
    if  PLOT and i % show_every == 0:
        out_np = torch_to_np(out)
        # print(out)
        # plot_image_grid([np.clip(out_np, 0, 1), 
        #                  np.clip(torch_to_np(out_avg), 0, 1)], factor=figsize, nrow=1, file_name=f"./output/denoise/denoise_{i:4d}.jpg")
        
    # Backtracking
    if i % show_every:
        if psrn_noisy - psrn_noisy_last < -5: 
            print('Falling back to previous checkpoint.')

            for new_param, net_param in zip(last_net, net.parameters()):
                net_param.data.copy_(new_param.cuda())

            return total_loss*0
        else:
            last_net = [x.detach().cpu() for x in net.parameters()]
            psrn_noisy_last = psrn_noisy
            
    i += 1

    return total_loss

p = get_params('net', net, net_input)
optimize(OPTIMIZER, p, closure, LR, num_iter)

Starting optimization with ADAM


  2%|▏         | 58/3000 [00:04<03:53, 12.57it/s] 


KeyboardInterrupt: 

In [ ]:
out_np = torch_to_np(net(net_input))
q = plot_image_grid([np.clip(out_np, 0, 1), img_np], factor=13,  file_name=f"./output/denoise/denoise_final.jpg");